# Question 1 - Alex Wiigs Stat 553 Final

**[10 marks] A data set in "DataSet1.csv" has observations for six variable $y_1,...,y_6$ from several groups (populations). Use this data set to perform hypothesis tests and discriminant analysis.**

In [109]:
library(tidyverse)

First we should load in the data and do some exploratory analysis. In particular, we should look at how many groups the data contains, how many variables they have and the total number of samples in the data set.

In [110]:
# Load data 
data <- read.csv("/Users/alexwiigs/Documents/Code/STAT 553 Final/DataSet1.csv")
head(data)
# shows the number of groups, variables and the total number of samples in the data
num_groups <- max(data[,1])
num_variables <- data |> select(-group) |> ncol()
n <- nrow(data)
print(paste("Number of groups in the data set:", num_groups))
print(paste("Numer of variables in the data set:", num_variables ))
print(paste("Total number of samples in the data set:", n))

,group,y1,y2,y3,y4,y5,y6
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,4.707869,4.278837,2.11668049,1.5988311,-0.3116434,-0.7212232
2,1,4.813089,3.943643,2.22508152,-1.3731751,-2.0235093,-0.5527398
3,1,7.598728,7.490672,0.72267853,0.1228908,-0.3456590,-0.9274571
4,1,7.707268,7.524177,-0.05352098,0.3776550,-0.7763228,-0.2856730
5,1,6.736214,5.065981,1.70665593,1.5999088,1.0570660,-0.1410927
6,1,4.928420,5.834367,-2.62305054,0.7447885,0.2523461,-0.6458310


[1] "Number of groups in the data set: 5"
[1] "Numer of variables in the data set: 6"
[1] "Total number of samples in the data set: 650"


In [111]:
# create function that gets number of samples in each group and print results
ni <- function(i) { data |> filter(group == i) |> select(-group) |> nrow() }
for (i in 1:num_groups) {print(paste("The number of samples in group:",i, ni(i))) }

[1] "The number of samples in group: 1 110"
[1] "The number of samples in group: 2 120"
[1] "The number of samples in group: 3 130"
[1] "The number of samples in group: 4 140"
[1] "The number of samples in group: 5 150"


Each group contains 10 more samples than the last. They are not of equal size.

**(a) Are the population covariance and matrices the same?**

The groups in the data set above each represent a sample from five different populations with respective mean and variance parameters $\mu_i$ and $\Sigma_i$ for $i = 1,...,5$. 

In order to test the equality of $5$ multivariate populations described above, set up the following hypothesis:  

$H_0: \Sigma_1 = ... = \Sigma_5$  
$H_1: \text{At least two of the variances are not equal}$  

Box's M-test is an appropriate statstic to use to test the hypothesis. M can be calculated as
$$M = \frac{\prod_{i=1}^k |S_i|^{(n_i-1)/2} } { |S_{pl}|^{ \sum_{i=1}^k (n_i-1)/2 }} $$
where
$$S_{pl} = \frac{\sum_{i=1}^k (n_i-1)S_i}{ \sum_{i=}^k n_i-k }= \frac{\textbf{E}}{v_{\textbf{E}}}$$
$S_i$ represents the sample covariance for group $i$, $n_i$ is the number of samples in group $i$ and $k$ is the total number of groups. $S_{pl}$ is the pooled covariance. functions that can calcuate these formulas are given below.

In [112]:
Si <- function(i) { data |> filter(group == i) |> select(-group) |> cov() }
Spl <- function(data) {
    numerator = matrix(0, nrow = ncol(data[,-1]), ncol = ncol(data[,-1]))
    for (i in 1:num_groups) { numerator = numerator + (ni(i) - 1) * Si(i) }
    denominator = nrow(data) - num_groups
    return(numerator / denominator)}
M <- function(data) {
    numerator = 1
    exponent = 0
    for (i in 1:num_groups) {numerator = numerator * det(Si(i))^( (ni(i) - 1) / 2 ) }
    for (i in 1:num_groups) {exponent = exponent + (ni(i) - 1) / 2}
    return( numerator / ( det(Spl(data))^exponent ) )}

Box (1949, 1950) gave a $\chi^2$ approximation for the distrubtion of $M$, which is:
$$u = -2(1-c_1)\ln(M) \sim \chi^2_{[\frac{1}{2}(k-1)p(p+1)]}$$
where 
$$c_1 = [\sum_{i=1}^k \frac{1}{n_i-1} - \frac{1}{\sum_{i=1}^k (n_1 - 1) }][ \frac{2p^2 + 3p - 1}{6(p+1)(k-1)}]$$
and $p$ is the number of variables.

In [113]:
c1 <- function(data) {
    term1 = 0
    term2 = 0
    for (i in 1:num_groups) {
        term1 = term1 + 1 / (ni(i) -1)
        term2 = term2 + (ni(i) -1)}
    value <- (term1 - 1 / term2) *
    (2 * num_variables^2 + 3 * num_variables - 1) / (6 * (num_variables + 1) * (num_groups - 1))
    return(value)}
df <- (num_groups - 1) * num_variables * ( num_variables + 1) / 2
print(paste("degrees of freedom for the test statistic approximation:", df))

[1] "degrees of freedom for the test statistic approximation: 84"


Thus, we will reject the null hypothesis in favor of the alternative if:
$$u_{\text{calculated}} \geq \chi^2_{\alpha = 0.05, 84} 

In [114]:
# Calculating the test statstic and rejection region. 
u <- -2 * (1 - c1(data)) * log(M(data))
rr <- qchisq(0.95, df)
p_value <- 1 - pchisq(u, df)
print(paste("Test statstic:", u))
print(paste("Rejection Region:", rr))
print(paste("The calcualted p-value for our observed test statistic is:", p_value))

[1] "Test statstic: 87.2703235703816"
[1] "Rejection Region: 106.394840242723"
[1] "The calcualted p-value for our observed test statistic is: 0.381932930940052"


Our calcualted test statstic, $u$ is not in the rejection region. Therefore we fail to reject the null hypothesis that the variances for the five group populations are different.

**(b) Are the population mean vectors the same?**

In order to preform analysis on the different groups, it is important to first check whether the difference in the groups is statistically significant. We create a MANOVA model for this.

$$\textbf{y}_{ij} = \mu_i + \epsilon_{ij}$$
$i = 1, ..., 5$, $j = 1,..., n_i$
$$\epsilon_{ij} \sim^{\text{i.i.d}} N_p(0, \Sigma )$$

By writing $\mu_i = \mu + \alpha_i$, where $\mu$ is the overall mean vector, and $\sum_{i=1}^5 \alpha_i =0$ we can state our null and alternative hypothesis as follows:  
  
$H_0: \alpha_1 = ... = \alpha_5 = 0$.     
$H_1:$ at least one $\alpha_i$ is not zero.

The Wilks' test statistic can preform this hypothesis test. The test statstic is 
$$\Lambda = \frac{|E|}{|E+H|} $$
where we will reject $H_0$ if $\Lambda \leq \Lambda_{\alpha, p, v_H, v_E}$. The matrix $E$ is a covariance matrix that represents the variance "within" the differnt groups and the matrix $H$ is a covariance matrix that repesents the variance "between" the each group and the overall average for all the groups. If $H$ is small relative to $E$, we will have little reason to reject the null hypothesis.

We calculate the "between" Matrix $H$ with the following theorem:
$$H = n \sum_{i=1}^k (\bar{\textbf{y}}_{i\cdot} - \bar{\textbf{y}}_{\cdot \cdot} )(\bar{\textbf{y}}_{i\cdot} - \bar{\textbf{y}}_{\cdot \cdot} )'$$

In [115]:
# Overall mean vector
overall_mean <- data |> select(-group) |> colSums() / ( n )
overall_mean <- overall_mean |> as.matrix()
# Calculate matrix where the rows are mean vectors for each of the five groups.
group_means <- data.frame()
for (i in 1:5) {
    temp_value <- data |> filter(group ==i) |> nrow()
    calculation <- data |> filter(group == i) |> select(-group) |> colSums() / temp_value
    group_means <- bind_rows(group_means, calculation)
    } 
group_means <- group_means |> as.matrix()

In [116]:
# Calculate the H matrix
H <- matrix(0, nrow = 6, ncol = 6)
for (i in 1:5) {
    ni <- data |> filter(group == i) |> select(-group) |> nrow()
    H <- H + ni * ( as.matrix(group_means[i,]) - overall_mean ) %*% 
    t( as.matrix(group_means[i,]) - overall_mean )  }
round(H,2)

,y1,y2,y3,y4,y5,y6
y1,25943.40,5165.60,29282.66,115.18,14.79,-3.07
y2,5165.60,9957.73,1464.52,-63.65,-44.85,67.41
y3,29282.66,1464.52,39234.27,214.80,118.35,4.02
y4,115.18,-63.65,214.80,8.62,0.77,-2.39
y5,14.79,-44.85,118.35,0.77,1.83,0.60
y6,-3.07,67.41,4.02,-2.39,0.60,1.60


Our formula for the "within" matrix $E$ is:
$$ E = \sum_{i=1}^k \sum_{j=1}^{n_i} (\textbf{y}_{ij} - \bar{ \textbf{y} }_{i \cdot} ) (\textbf{y}_{ij} - \bar{ \textbf{y} }_{i\cdot} )'$$ 

In [117]:
# Calcualte the E matrix
E <- matrix(0, nrow = 6, ncol = 6)
for (i in 1:5) {
    for ( j in 1:nrow(filter(data, group == i))) {
        y_ij <- data |> filter(group == i) |> select(-group) |> slice(j)
        E <- E + (t(y_ij) - group_means[i,] ) %*% t(t(y_ij) - group_means[i,] ) }}
round(E,2)

,y1,y2,y3,y4,y5,y6
y1,1203.34,563.79,691.20,14.37,-6.19,-4.71
y2,563.79,1304.08,-691.18,27.51,-5.94,-29.81
y3,691.20,-691.18,2020.05,-61.92,-15.48,11.81
y4,14.37,27.51,-61.92,585.53,32.29,-12.27
y5,-6.19,-5.94,-15.48,32.29,570.52,22.14
y6,-4.71,-29.81,11.81,-12.27,22.14,315.86


Now that we have calculated the matrices we can caulate the test statistic.

In [118]:
Lambda <- det(E) / det( E + H)
Lambda

[1] 0.0005536129

Using Roy's test Statstic:

In [119]:
eigen(solve(E) %*% H)

eigen() decomposition
$values
[1] 2.795345e+01 1.440113e+01 2.996200e+00 1.366330e-02 2.394548e-17
[6] 4.185473e-18

$vectors
            [,1]         [,2]        [,3]         [,4]         [,5]
[1,] -0.37920414 -0.609389486 -0.77091750 -0.003092889  0.026797271
[2,] -0.46996577  0.694801095  0.08250642  0.010330432 -0.009396429
[3,] -0.79363585  0.377619038  0.62199226 -0.002508665 -0.023395291
[4,] -0.06263693  0.018975341  0.10147400  0.874695980  0.250666127
[5,] -0.03098299  0.008280308  0.03852232 -0.110802559  0.766819936
[6,] -0.02439659  0.053582908 -0.01495283 -0.471706645  0.589745668
             [,6]
[1,] -0.011705911
[2,] -0.002253831
[3,]  0.010938265
[4,]  0.081280315
[5,] -0.866183197
[6,]  0.492806681


In [120]:
lambda_1 <- eigen(solve(E) %*% H)$values |> max()
theta <- lambda_1 / (lambda_1 + 1)
theta

[1] 0.9654618

**(c)** Find discriminant functions and use the first two discriminant functions to show the data set.

In [121]:
vectors <- eigen(solve(E) %*% H)$vectors
values <- eigen(solve(E) %*% H )$values
